## Create Ballanced Data Set

In [2]:
import time
import numpy as np
import pandas as pd
import random as rand
pd.options.display.max_rows = 25
pd.options.display.max_columns  = 25

from pandas.api.types import CategoricalDtype

from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE # doctest: +NORMALIZE_WHITESPACE
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn import model_selection

use_smote = False
binaryClass = True

inputfile = 'CKME136X10_2018_Data_CTF.csv'
if binaryClass:
    outputfile_train_O = 'CKME136X10_2018_Data_CTFB_B_O_Train.csv'
    outputfile_train_U = 'CKME136X10_2018_Data_CTFB_B_U_Train.csv'
    outputfile_test = 'CKME136X10_2018_Data_CTFB_B_Test.csv'
else:
    outputfile_train_O = 'CKME136X10_2018_Data_CTFB_M_O_Train.csv'
    outputfile_train_U = 'CKME136X10_2018_Data_CTFB_M_U_Train.csv'
    outputfile_test = 'CKME136X10_2018_Data_CTFB_M_Test.csv'


In [3]:
#load data
df = pd.read_csv(inputfile, engine = 'python')

FileNotFoundError: [Errno 2] No such file or directory: 'CKME136X10_2018_Data_CTF.csv'

In [ ]:
df_columns = df.columns[:len(df.columns) -1]
print(df_columns)

In [ ]:
#check for nun numeric values
print("Number of Null values: {}".format(df.isnull().sum().sum()))
print()
print("Number of NaN: {0}".format(df.isna().sum().sum()))
print()
print("Number of Non Numeric: {}".format(df[df.columns].apply(lambda x: x.astype(str).str.contains('[^0-9]')).sum().sum()))
print()
print('Information on the imported data')
print(df.info())

In [ ]:
# convert to the correct type
df = df.astype('category')
df['C_YEAR'] = df['C_YEAR'].astype(CategoricalDtype(ordered=True))
df['C_MNTH'] = df['C_MNTH'].astype(CategoricalDtype(ordered=True))
df['C_WDAY'] = df['C_WDAY'].astype(CategoricalDtype(ordered=True))
df['C_HOUR'] = df['C_HOUR'].astype(CategoricalDtype(ordered=True))
df['C_VEHS'] = df['C_VEHS'].astype('int')
df['V_YEAR'] = df['V_YEAR'].astype(CategoricalDtype(ordered=True))
df['P_PSN'] = df['P_PSN'].astype(CategoricalDtype(ordered=True))
df['P_AGE'] = df['P_AGE'].astype('int')
df['P_ISEV'] = df['P_ISEV'].astype('int')
print(df.info())

## Split Training and Test set 70/30 split

In [ ]:
## Split Training and Test set 70/30 split, so we don't bleed information to test set
#Split between data and class
Y = df[df.columns[-1]].copy()
if binaryClass:
    Y.replace(to_replace = 1, value = 0, inplace = True)
    Y.replace(to_replace = 2, value = 1, inplace = True)
    Y.replace(to_replace = 3, value = 1, inplace = True)
else:
    Y.replace(to_replace = 1, value = 0, inplace = True)
    Y.replace(to_replace = 2, value = 1, inplace = True)
    Y.replace(to_replace = 3, value = 2, inplace = True)

X = df[df.columns[0:df.columns.size -1]].copy()

X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.3, stratify=Y)

In [ ]:
print(Y_train.groupby(Y_train).size())
print(Y_test.groupby(Y_test).size())


In [ ]:
X_test.info()

In [ ]:
#Testing Set
df_test = X_test.copy()
df_test['P_ISEV'] = Y_test.copy()

### over sample using smote

In [ ]:
if (use_smote):
# use somte to over sample the minority class
    sm = SMOTE(random_state=12, ratio = 'minority')
    x_res, y_res = sm.fit_sample(X_train, Y_train)
    print('Resampled dataset shape %s' % Counter(y_res))
    
    df_train_over = pd.DataFrame(data=x_res, columns = df_columns)
    df_train_over['P_ISEV'] = y_res

    # convert to the correct type

    #not sure why converting to category still leave fold values, lets convert to int first
    df_train_over = df_train_over.astype('int')
    df_train_over = df_train_over.astype('category')
    df_train_over['C_YEAR'] = df_train_over['C_YEAR'].astype(CategoricalDtype(ordered=True))
    df_train_over['C_MNTH'] = df_train_over['C_MNTH'].astype(CategoricalDtype(ordered=True))
    df_train_over['C_WDAY'] = df_train_over['C_WDAY'].astype(CategoricalDtype(ordered=True))
    df_train_over['C_HOUR'] = df_train_over['C_HOUR'].astype(CategoricalDtype(ordered=True))
    df_train_over['C_VEHS'] = df_train_over['C_VEHS'].astype('int')
    df_train_over['V_YEAR'] = df_train_over['V_YEAR'].astype(CategoricalDtype(ordered=True))
    df_train_over['P_PSN'] = df_train_over['P_PSN'].astype(CategoricalDtype(ordered=True))
    df_train_over['P_AGE'] = df_train_over['P_AGE'].astype('int')
    df_train_over['P_ISEV'] = df_train_over['P_ISEV'].astype('int')
    print(df_train_over.shape)
    df_train_over.head(3)

### under sample using smote

In [ ]:
if (use_smote):
    # use somte to under sample the minority class
    sm = SMOTE(random_state=12, ratio = 'not minority')
    x_res, y_res = sm.fit_sample(X_train, Y_train)
    print('Resampled dataset shape %s' % Counter(y_res))
    
    df_train_under = pd.DataFrame(data=x_res, columns = df_columns)
    df_train_under['P_ISEV'] = y_res

    df_train_under = df_train_under.astype('int')
    df_train_under = df_train_under.astype('category')
    df_train_under['C_YEAR'] = df_train_under['C_YEAR'].astype(CategoricalDtype(ordered=True))
    df_train_under['C_MNTH'] = df_train_under['C_MNTH'].astype(CategoricalDtype(ordered=True))
    df_train_under['C_WDAY'] = df_train_under['C_WDAY'].astype(CategoricalDtype(ordered=True))
    df_train_under['C_HOUR'] = df_train_under['C_HOUR'].astype(CategoricalDtype(ordered=True))
    df_train_under['C_VEHS'] = df_train_under['C_VEHS'].astype('int')
    df_train_under['V_YEAR'] = df_train_under['V_YEAR'].astype(CategoricalDtype(ordered=True))
    df_train_under['P_PSN'] = df_train_under['P_PSN'].astype(CategoricalDtype(ordered=True))
    df_train_under['P_AGE'] = df_train_under['P_AGE'].astype('int')
    df_train_under['P_ISEV'] = df_train_under['P_ISEV'].astype('int')

    print(df_train_under.shape)
    df_train_under.head(3)

### over sample using RamdomOverSampler

In [ ]:
if (not use_smote):
    ros = RandomOverSampler(random_state=0, ratio = 'minority')
    X_resampled_ros, y_resampled_ros = ros.fit_sample(X_train, Y_train)
    print(sorted(Counter(y_resampled_ros).items()))
    
    df_train_under = pd.DataFrame(data=X_resampled_ros, columns = df_columns)
    df_train_under['P_ISEV'] = y_resampled_ros

    df_train_under = df_train_under.astype('int')
    df_train_under = df_train_under.astype('category')
    df_train_under['C_YEAR'] = df_train_under['C_YEAR'].astype(CategoricalDtype(ordered=True))
    df_train_under['C_MNTH'] = df_train_under['C_MNTH'].astype(CategoricalDtype(ordered=True))
    df_train_under['C_WDAY'] = df_train_under['C_WDAY'].astype(CategoricalDtype(ordered=True))
    df_train_under['C_HOUR'] = df_train_under['C_HOUR'].astype(CategoricalDtype(ordered=True))
    df_train_under['C_VEHS'] = df_train_under['C_VEHS'].astype('int')
    df_train_under['V_YEAR'] = df_train_under['V_YEAR'].astype(CategoricalDtype(ordered=True))
    df_train_under['P_PSN'] = df_train_under['P_PSN'].astype(CategoricalDtype(ordered=True))
    df_train_under['P_AGE'] = df_train_under['P_AGE'].astype('int')
    df_train_under['P_ISEV'] = df_train_under['P_ISEV'].astype('int')

    print(df_train_under.shape)
    df_train_under.head(3)

In [ ]:
### under sample non minority classes

In [ ]:
if (not use_smote):
    # under sample using Ramdom Under Sampler, much faster than smote
    rus = RandomUnderSampler(random_state=0, ratio = 'not minority')
    X_resampled_rus, y_resampled_rus = rus.fit_sample(X_train, Y_train)
    print(sorted(Counter(y_resampled_rus).items()))

    df_train_under = pd.DataFrame(data=X_resampled_rus, columns = df_columns)
    df_train_under['P_ISEV'] = y_resampled_rus

    df_train_under = df_train_under.astype('int')
    df_train_under = df_train_under.astype('category')
    df_train_under['C_YEAR'] = df_train_under['C_YEAR'].astype(CategoricalDtype(ordered=True))
    df_train_under['C_MNTH'] = df_train_under['C_MNTH'].astype(CategoricalDtype(ordered=True))
    df_train_under['C_WDAY'] = df_train_under['C_WDAY'].astype(CategoricalDtype(ordered=True))
    df_train_under['C_HOUR'] = df_train_under['C_HOUR'].astype(CategoricalDtype(ordered=True))
    df_train_under['C_VEHS'] = df_train_under['C_VEHS'].astype('int')
    df_train_under['V_YEAR'] = df_train_under['V_YEAR'].astype(CategoricalDtype(ordered=True))
    df_train_under['P_PSN'] = df_train_under['P_PSN'].astype(CategoricalDtype(ordered=True))
    df_train_under['P_AGE'] = df_train_under['P_AGE'].astype('int')
    df_train_under['P_ISEV'] = df_train_under['P_ISEV'].astype('int')

    print(df_train_under.shape)
    df_train_under.head(3)

In [ ]:
df_test.info()

In [ ]:
df_train_over.to_csv(outputfile_train_O, encoding='utf-8', index=False)
df_train_under.to_csv(outputfile_train_U, encoding='utf-8', index=False)
df_test.to_csv(outputfile_test, encoding='utf-8', index=False)